In [9]:
import mysql.connector
import pandas as pd
from getpass import getpass

In [4]:
password = getpass()

In [14]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password=password,
  port="3306"
)

In [19]:
df = pd.read_csv('../../production/action-server/datasets/drugs_dataset.csv')

In [20]:
df.columns

Index(['medicine', 'vitamin', 'before-taking', 'cautions', 'contraindications',
       'directions', 'dosage', 'faq', 'fda-approval', 'interactions',
       'patient-advice', 'patient-counseling', 'patient-education',
       'pharmacokinetics', 'pharmacology', 'pregnancy', 'preparations',
       'side-effects', 'storage', 'uses', 'warnings', 'what-to-avoid'],
      dtype='object')

In [21]:
df_selected = df[['medicine', 'dosage', 'interactions', 'side-effects', 'uses', 'warnings', 'what-to-avoid']]

In [22]:
df_final = df_selected.replace(r'\n+', '\n', regex=True)

In [27]:
df_final.astype('str').applymap(lambda x: len(x)).max()

medicine            42
dosage           18494
interactions     18864
side-effects      8891
uses              8194
warnings         12408
what-to-avoid     2056
dtype: int64

In [28]:
'''
To-Do:
Create a database 
Create a drugs table (according to the max length of each column) in the database
Create a lab table (according to the max length of each column) in the database
Load drugs to the drug table
Load labs to the lab table

Sample code:
LOAD DATA INFILE 'file_name'
INTO TABLE table_name
FIELDS TERMINATED BY ',' 
ENCLOSED BY '"'
LINES TERMINATED BY '\n'
IGNORE 1 ROWS

Use query to get information from table 

Sample code:
SELECT dosage
FROM drugs
WHERE medicine='acetaminophen';
'''

'\nTo-Do:\nCreate a database \nCreate a drugs table (according to the max length of each column) in the database\nCreate a lab table (according to the max length of each column) in the database\nLoad drugs to the drug table\nLoad labs to the lab table\n\nSample code:\nLOAD DATA INFILE \'file_name\'\nINTO TABLE table_name\nFIELDS TERMINATED BY \',\' \nENCLOSED BY \'"\'\nLINES TERMINATED BY \'\n\'\nIGNORE 1 ROWS\n\nUse query to get information from table \n\nSample code:\nSELECT dosage\nFROM drugs\nWHERE medicine=\'acetaminophen\';\n'